In [ ]:
!pip install gcsfs

In [7]:
import pandas as pd
from google.cloud import storage
import gcsfs
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
image_names = !ls ../raw_data/images_training_rev1/ #loading images_names.jpg to list
df_image_names = pd.DataFrame(image_names)  
df_image_names = df_image_names.rename(columns={0:'image'}) 
df_training_solutions_rev1 = pd.read_csv('../raw_data/training_solutions_rev1.csv')
#loading y dataframe

In [3]:
df_training_solutions_rev1['image'] = df_image_names['image']
df_X_y = df_training_solutions_rev1.drop(columns=['GalaxyID'])
df_X_y.head(2) #final dataframe with y and image_names.jpg

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.0,...,0.279952,0.138445,0.00000,0.0,0.092886,0.0,0.0,0.0,0.325512,100008.jpg
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.0,...,0.000000,0.131378,0.45995,0.0,0.591328,0.0,0.0,0.0,0.000000,100023.jpg


In [4]:
df_X_y_sample = df_X_y.sample(400)
df_X_y_sample.shape

(400, 38)

In [5]:
df_X_y_train = df_X_y_sample[:200]
df_X_y_test = df_X_y_sample[200:]
df_X_y_train.shape

(200, 38)

In [ ]:
# Matching the dataframe with the images in folder
df = df.merge(df_Galaxy_ID, on='GalaxyID', how='inner')
df.shape

In [ ]:
N = 400
df_X_y_sample = df_X_y.sample(N)
df_X_y_train = df_X_y_sample[:int(N/2)]
df_X_y_test = df_X_y_sample[int(N/2):]
df_X_y_test.shape

In [8]:
import tensorflow as tf

def load_image(image):
    '''returns one image'''
    folder = f'../raw_data/images_training_rev1/{image}'
    img = Image.open(folder)
    img_array = np.array(img)
    return np.resize(img_array, (224,224,3))
    
def load_images(df):
    '''returns array of images'''
    img_list = []
    for _, row in df.iterrows():
        img = load_image(row["image"])
        img_list.append(img) 
    return np.stack(img_list)

class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, df, batch_size=64, shuffle=True, ):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        '''returns number of minibatches per epoch'''
        return len(self.indices) // self.batch_size
    
    def on_epoch_end(self):
        '''shuffles the indices '''
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
            
    def __get_X_image(self, df):
        '''returns images'''
        X = load_images(df)
        return X 
             
    def __get_Y(self, df):
        '''returns y'''
        return np.array(df.drop(columns=["image"]))
    
    def _get_data(self, batch):
        '''returns batch of images and y'''
        df_batch = self.df.query("index in @batch") 
        X = self.__get_X_image(df_batch)
        y = self.__get_Y(df_batch)
        return X, y
    
    def __getitem__(self, index):
        '''creates batches and returns final X and y'''
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        X, y = self._get_data(batch)
        return X, y

In [9]:
datagenerator_train = DataGenerator(df=df_X_y_train, shuffle=True)
datagenerator_test = DataGenerator(df=df_X_y_test, shuffle=True)

In [ ]:
# import keras 
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Concatenate, Conv2D, Flatten, MaxPool2D
from keras import Sequential
from tensorflow.keras.models import Model

model = Sequential()
model.add(Input(shape=(224,224,3)))

model.add(Conv2D(filters=254,kernel_size=(1,1), activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(1,1), activation='relu'))
# model.add(Conv2D(16, 16, activation='relu'))
# model.add(Conv2D(2, 2, activation='relu'))
model.add(MaxPool2D(pool_size=(8,8),strides=(8,8)))

model.add(Flatten())
#model.add(Dense(200, activation='linear'))
model.add(Dense(37, activation='linear'))


In [10]:
import tensorflow.compat.v1 as tf
from keras import models
from keras import layers
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Concatenate, Conv2D, Flatten, MaxPool2D
from keras import Sequential
from keras.models import Model
tf.disable_v2_behavior()

def fire_module(inputs,s1x1,e1x1,e3x3,name="fire"):
    w_init = tf.truncated_normal_initializer(mean=0.0, stddev=(1.0/int(inputs.shape[2])))
    
    with tf.variable_scope(name):
        #squeeze layer
        squeeze_out = tf.layers.conv2d(inputs,filters=s1x1,kernel_size=1,strides=1,padding="VALID",kernel_initializer=w_init)
        relu_sq = tf.nn.relu(squeeze_out)
    
        #expand layer
        k1_exp = tf.layers.conv2d(relu_sq,filters=e1x1,kernel_size=1,strides=1,padding="VALID",kernel_initializer=w_init)
        k1_relu = tf.nn.relu(k1_exp)
        k3_exp = tf.layers.conv2d(relu_sq,filters=e3x3,kernel_size=3,strides=1,padding="SAME",kernel_initializer=w_init)
        k3_relu = tf.nn.relu(k3_exp)
        
        return tf.concat([k1_relu,k3_relu],axis=3)

"""
General Convolution Operation
"""
def general_conv(inputs,filters,kernel,stride=1,padding='VALID',name="conv",relu = True,weight="Xavier"):
    if str(weight) == str("Xavier"):
        w_init = tf.truncated_normal_initializer(mean=0.0,stddev=(1.0/int(inputs.shape[2])))
    else:
        w_init = tf.truncated_normal_initializer(mean=0.0,stddev=0.01)
        
    with tf.variable_scope(name):
        conv = tf.layers.conv2d(inputs,filters,kernel,stride,padding,kernel_initializer=w_init)
        if relu == True:
                conv = tf.nn.relu(conv)
        return conv

"""
SqueezeNet Class Definition
"""
class SqueezeNet:
    
    def __init__(self,input_shape,out_classes,lr_rate,train):
        self.lr_rate = tf.placeholder(tf.float32,name="lr_rate")
        self.out_classes = out_classes
        self.inputs = tf.placeholder(tf.float32,shape=(None,input_shape[0],input_shape[1],input_shape[2]))
        self.labels = tf.placeholder(tf.float32,shape=(None,self.out_classes))
        self.loss_v0,self.loss_v0_res,self.loss_v1 = self.model_loss(self.inputs,self.labels,train)     
        self.v0_opt,self.v0_res_opt,self.v1_opt = self.model_opti(self.loss_v0,self.loss_v0_res,self.loss_v1,self.lr_rate)
        
    #Model Definition of SqueezeNet V0
    def model_arc_v0(self,inputs,train,reuse=False):
        
        with tf.variable_scope("squeezenet_v0",reuse=reuse):
            conv1 = general_conv(inputs,filters=96,kernel=7,stride=2,padding="SAME",name="conv1",relu=True,weight="Xavier")
            pool1 = tf.layers.max_pooling2d(conv1,pool_size=3,strides=2,name="pool1")
            
            fire2 = fire_module(pool1,16,64,64,name="fire2")
            fire3 = fire_module(fire2,16,64,64,name="fire3")
            fire4 = fire_module(fire3,32,128,128,name="fire4")
        
            pool2 = tf.layers.max_pooling2d(fire4,pool_size=3,strides=2,name="pool2")
            
            fire5 = fire_module(pool2,32,128,128,name="fire5")
            fire6 = fire_module(fire5,48,192,192,name="fire6")
            fire7 = fire_module(fire6,48,192,192,name="fire7")
            fire8 = fire_module(fire7,64,256,256,name="fire8")
        
            pool3 = tf.layers.max_pooling2d(fire8,pool_size=3,strides=2,name="pool3")
        
            fire9 = fire_module(pool3,64,256,256,name="fire9")
            drop = tf.layers.dropout(fire9,rate=0.5,training=train)
        
            conv10 = general_conv(drop,filters=200,kernel=1,stride=1,padding="SAME",name="conv10",relu=True,weight="Gaussian")
        
            avg_pool = tf.layers.average_pooling2d(conv10,pool_size=13,strides=1,name="pool_end")
            pool_shape = tf.shape(avg_pool)
            logits = tf.reshape(avg_pool,shape=(pool_shape[0],pool_shape[3]))
                    
            return logits
        
    #Model Definiton of SqueezeNet V1    
    def model_arc_v1(self,inputs,train,reuse=False):
        
        with tf.variable_scope("squeezenet_v1",reuse=reuse):
            conv1 = general_conv(inputs,filters=64,kernel=3,stride=2,padding="SAME",name="conv1",relu=True,weight="Xavier")
            pool1 = tf.layers.max_pooling2d(conv1,pool_size=3,strides=2,name="pool1")
            
            fire2 = fire_module(pool1,16,64,64,name="fire2")
            fire3 = fire_module(fire2,16,64,64,name="fire3")
            
            pool2 = tf.layers.max_pooling2d(fire3,pool_size=3,strides=2,name="pool2")
            
            fire4 = fire_module(pool2,32,128,128,name="fire4")
            fire5 = fire_module(fire4,32,128,128,name="fire5")
            
            pool3 = tf.layers.max_pooling2d(fire5,pool_size=3,strides=2,name="pool3")
            
            fire6 = fire_module(pool3,48,192,192,name="fire6")
            fire7 = fire_module(fire6,48,192,192,name="fire7")
            fire8 = fire_module(fire7,64,256,256,name="fire8")       
            fire9 = fire_module(fire8,64,256,256,name="fire9")
            drop = tf.layers.dropout(fire9,rate=0.5,training=train)
        
            conv10 = general_conv(drop,filters=200,kernel=1,stride=1,padding="SAME",name="conv10",relu=True,weight="Gaussian")
        
            avg_pool = tf.layers.average_pooling2d(conv10,pool_size=13,strides=1,name="pool_end")
        
            pool_shape = tf.shape(avg_pool)
            logits = tf.reshape(avg_pool,shape=(pool_shape[0],pool_shape[3]))
        
            return logits
        
        
    #Model Definition of SqueezeNet V0 Residual     
    def model_arc_v0_res(self,inputs,train,reuse=False):
        
        with tf.variable_scope("squeezenet_v0_res",reuse=reuse):
            conv1 = general_conv(inputs,filters=96,kernel=7,stride=2,padding="SAME",name="conv1",relu=True,weight="Xavier")
            pool1 = tf.layers.max_pooling2d(conv1,pool_size=3,strides=2,name="pool1")
            
            fire2 = fire_module(pool1,16,64,64,name="fire2")
            fire3 = fire_module(fire2,16,64,64,name="fire3")
            
            bypass_23 = tf.add(fire2,fire3,name="bypass_23")
            
            fire4 = fire_module(bypass_23,32,128,128,name="fire4")
            pool2 = tf.layers.max_pooling2d(fire4,pool_size=3,strides=2,name="pool2")
            fire5 = fire_module(pool2,32,128,128,name="fire5")
            
            bypass_45 = tf.add(pool2,fire5,name="bypass_45")
            
            fire6 = fire_module(bypass_45,48,192,192,name="fire6")
            fire7 = fire_module(fire6,48,192,192,name="fire7")
            fire8 = fire_module(fire7,64,256,256,name="fire8")
        
            pool3 = tf.layers.max_pooling2d(fire8,pool_size=3,strides=2,name="pool3")
        
            fire9 = fire_module(pool3,64,256,256,name="fire9")
            
            bypass_89 = tf.add(pool3,fire9,name="bypass_89")
            
            drop = tf.layers.dropout(bypass_89,rate=0.5,training=train)
        
            conv10 = general_conv(drop,filters=200,kernel=1,stride=1,padding="SAME",name="conv10",relu=True,weight="Gaussian")
        
            avg_pool = tf.layers.average_pooling2d(conv10,pool_size=13,strides=1,name="pool_end")
        
            pool_shape = tf.shape(avg_pool)
            logits = tf.reshape(avg_pool,shape=(pool_shape[0],pool_shape[3]))
        
            return logits


    #Function to calculate loss 
    def model_loss(self,inputs,label,train):
        logits_v0 = self.model_arc_v0(inputs,train)
        logits_v0_res = self.model_arc_v0_res(inputs,train)
        logits_v1 = self.model_arc_v1(inputs,train)
        
        loss_v0 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_v0,labels=label))
        loss_v0_res = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_v0_res,labels=label))
        loss_v1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_v1,labels=label))
        
        return loss_v0,loss_v0_res,loss_v1
    
    #Function to calculate prediction form models
    def model_prediction(self,inputs,train):
        logits_v0 = self.model_arc_v0(inputs,train,True)
        logits_v0_res = self.model_arc_v0_res(inputs,train,True)
        logits_v1 = self.model_arc_v1(inputs,train,True)
        
        predict_v0 = tf.nn.softmax(logits_v0)
        predict_v0_res = tf.nn.softmax(logits_v0_res)
        predict_v1 = tf.nn.softmax(logits_v1)
        
        return predict_v0,predict_v0_res,predict_v1
    
    #Function to optimize the models. 
    def model_opti(self,loss_v0,loss_v0_res,loss_v1,lr_rate):
        
        train_vars = tf.trainable_variables()
        v0_vars = [var for var in train_vars if var.name.startswith('squeezenet_v0')]
        v0_res_vars = [var for var in train_vars if var.name.startswith('squeezenet_v0_res')]
        v1_vars = [var for var in train_vars if var.name.startswith('squeezenet_v1')]        
        
        #Using Adam Optimizer 
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            v0_train_opt = tf.train.AdamOptimizer(lr_rate).minimize(loss_v0,var_list=v0_vars)
            v0_res_train_opt = tf.train.AdamOptimizer(lr_rate).minimize(loss_v0_res,var_list=v0_res_vars)
            v1_train_opt = tf.train.AdamOptimizer(lr_rate).minimize(loss_v1,var_list=v1_vars)
            
        return v0_train_opt, v0_res_train_opt, v1_train_opt
        

Instructions for updating:
non-resource variables are not supported in the long term


In [33]:

def fire_module(inputs,s1x1,e1x1,e3x3,name="fire"):
    w_init = tf.truncated_normal_initializer(mean=0.0, stddev=(1.0/int(inputs.shape[2])))
    
    with tf.variable_scope(name):
        #squeeze layer
        squeeze_out = tf.layers.conv2d(inputs,filters=s1x1,kernel_size=1,strides=1,padding="VALID",kernel_initializer=w_init)
        relu_sq = tf.nn.relu(squeeze_out)
    
        #expand layer
        k1_exp = tf.layers.conv2d(relu_sq,filters=e1x1,kernel_size=1,strides=1,padding="VALID",kernel_initializer=w_init)
        k1_relu = tf.nn.relu(k1_exp)
        k3_exp = tf.layers.conv2d(relu_sq,filters=e3x3,kernel_size=3,strides=1,padding="SAME",kernel_initializer=w_init)
        k3_relu = tf.nn.relu(k3_exp)
        
        return tf.concat([k1_relu,k3_relu],axis=3)

    
def squeezenet(inputs):
            
        
            conv1 = tf.layers.conv2d((224,224,3),filters=96,strides=2, kernel_size=(2,2),name="conv1")
            pool1 = tf.layers.max_pooling2d(conv1,pool_size=3,strides=2,name="pool1")
            
            fire2 = fire_module(pool1,16,64,64,name="fire2")
            fire3 = fire_module(fire2,16,64,64,name="fire3")
            
            bypass_23 = tf.add(fire2,fire3,name="bypass_23")
            
            fire4 = fire_module(bypass_23,32,128,128,name="fire4")
            pool2 = tf.layers.max_pooling2d(fire4,pool_size=3,strides=2,name="pool2")
            fire5 = fire_module(pool2,32,128,128,name="fire5")
            
            bypass_45 = tf.add(pool2,fire5,name="bypass_45")
            
            fire6 = fire_module(bypass_45,48,192,192,name="fire6")
            fire7 = fire_module(fire6,48,192,192,name="fire7")
            fire8 = fire_module(fire7,64,256,256,name="fire8")
        
            pool3 = tf.layers.max_pooling2d(fire8,pool_size=3,strides=2,name="pool3")
        
            fire9 = fire_module(pool3,64,256,256,name="fire9")
            
            bypass_89 = tf.add(pool3,fire9,name="bypass_89")
            
            drop = tf.layers.dropout(bypass_89,rate=0.5,training=train)
        
            conv10 = general_conv(drop,filters=200,kernel=1,stride=1,padding="SAME",name="conv10",relu=True,weight="Gaussian")
        
            avg_pool = tf.layers.average_pooling2d(conv10,pool_size=13,strides=1,name="pool_end")
        
            pool_shape = tf.shape(avg_pool)
            logits = tf.reshape(avg_pool,shape=(pool_shape[0],pool_shape[3]))
            
            model = logits.compile(loss='mse',  #loss a voir
              optimizer='adam',
              metrics='mse')
        
            return model


In [34]:
model = squeezenet((224,224,3))

ValueError: Layer conv1 expects 1 input(s), but it received 3 input tensors. Inputs received: [<tf.Tensor 'Const_6:0' shape=() dtype=int32>, <tf.Tensor 'Const_7:0' shape=() dtype=int32>, <tf.Tensor 'Const_8:0' shape=() dtype=int32>]

In [12]:
model.summary()

AttributeError: 'function' object has no attribute 'summary'

In [13]:
model.compile(loss='mse',  #loss a voir
              optimizer='adam',
              metrics='mse')

AttributeError: 'function' object has no attribute 'compile'

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=7, restore_best_weights=True )

In [15]:
model.fit(datagenerator_train, batch_size=2, epochs=100, callbacks=es, verbose=1, validation_data=datagenerator_test)

AttributeError: 'function' object has no attribute 'fit'